In [2]:
import ffmpeg

input_file = "/Users/ydubey/Desktop/transcription/241118_1304.mp3"
output_file = "/Users/ydubey/Desktop/transcription/241118_1304.wav"

try:
    stream = ffmpeg.input(input_file)
    stream = ffmpeg.output(stream, output_file)
    ffmpeg.run(stream)
    print("Conversion successful")
except ffmpeg.Error as e:
    print(f"Error: {e.stderr.decode()}")

Conversion successful


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/opt/anaconda3/envs/transcription --cc=arm64-apple-darwin20.0.0-clang --ar=arm64-apple-darwin20.0.0-ar --nm=arm64-apple-darwin20.0.0-nm --ranlib=arm64-apple-darwin20.0.0-ranlib --strip=arm64-apple-darwin20.0.0-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libdav1d --enable-zlib --enable-libaom --enable-pic --enable-shared --disable-static --disable-gpl --enable-version3 --disable-sdl2 --enable-libopenh264 --enable-libopus --enable-libmp3lame --enable-libopenjpeg --enable-libvorbis --enable-pthreads --enable-libtesseract --enable-libvpx --enable-librsvg
  libavutil      58. 29.100 / 58. 29.100
  libavcodec     60. 31.102 / 60. 31.102
  libavformat    60. 16.100 

In [14]:
import pandas as pd
import torch
from pyannote.audio import Pipeline
import whisper

# Initialize Pyannote pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="YOUR_GITHUB_TOKEN")

# Load audio file
audio_file = "/Users/ydubey/Desktop/transcription/241118_1304.wav"

# Check if MPS is available and set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

# Send pipeline to the appropriate device
pipeline.to(device)

# Perform diarization
diarization = pipeline(audio_file)

# Load Whisper model and transcribe audio
model = whisper.load_model("base.en")
result = model.transcribe(audio_file)

# Process diarization and transcription results
transcript = {}
used_segments = set()  # Keep track of used segments

for segment, _, speaker in diarization.itertracks(yield_label=True):
    text = ""
    for section in result["segments"]:
        if (section["end"] >= segment.start and section["start"] <= segment.end and 
            section["id"] not in used_segments):
            text += " " + section["text"]
            used_segments.add(section["id"])
    
    if text.strip():  # Only add if there's non-empty text
        if speaker not in transcript:
            transcript[speaker] = []
        transcript[speaker].append((segment.start, segment.end, text.strip()))

# Create DataFrame
df = pd.DataFrame([(speaker, start, end, text) 
                   for speaker, segments in transcript.items() 
                   for start, end, text in segments],
                  columns=["Speaker", "Start", "End", "Text"])

# Sort by start time and save to CSV
df.sort_values(by="Start").reset_index(drop=True).to_csv("transcription_results.csv", index=False)

print("Transcription results saved to 'transcription_results.csv")

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1. Bad things might happen unless you revert torch to 1.x.


INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/opt/anaconda3/envs/transcription/lib/python3.10/site-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will exec

Using mps


INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/opt/anaconda3/envs/transcription/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary

Transcription results saved to 'transcription_results_updated.csv'.
